# Bodyfat Calculator

Group member: Hongyi Jin, Wanwan Su, Yingjing Jiang

Body fat percentage, a measure of obesity, plays an important role in various health outcomes such as life expectancy, prognosis for disease, healthcare costs, and the general well-being of individuals. Body fat percentage is helpful for people to have a basic knowledge of their health condiction. However, accurate measurement of body fat is often costly and inconvenient and it is desirable to have easy methods of estimating body fat that are not inconvenient/costly. 

In this module, we are going to come up with a simple and robust method to estimate percentage of body fat based on a real data set of 252 men with measurements of their percentage of body fat and other body measurements.

## 1.Data Processing

### 1.1 Data Description

1. Health assessment method support: Accordind to some popular books, estimating the percentage of body fat is at least a part, considered to assess body health. The books show that age, skin-fold measurements and body circumference measurements are used to estimate body fat.
2. Fact: Percentage of body fat for an individual can be estimated once body density has been determined. Therefore, we would try to figure out if body-fat percentage is highly-related with body density.
3. Equation and  Definition:
Siri (1956)) assume that the body consists of two components - lean body tissue and fat tissue.
D = 1/[(A/a) + (B/b)], B = (1/D)*[ab/(a-b)] - [b/(a-b)], 
D = Body Density (gm/cm^3) 
A = proportion of lean body tissue 
B = proportion of fat tissue (A+B=1) 
a = density of lean body tissue (gm/cm^3) 
b = density of fat tissue (gm/cm^3), 

"Siri's equation":  
Percentage of Body Fat (i.e. 100*B) = 495/D - 450.  
From Siri's equation, we get the relationship between body fat percentage and  body density.
4. There is a method to calculate body density accurately based on determining body volume by underwater submersion. But it is hard for doctors who want to and easily quickly determine a patient’s body fat percentage based on commonly available measurements to use.
5. Available measurements and Data Description
Age (years)  
Weight (lbs)  
Height (inches)  
Adioposity (bmi)
Neck circumference (cm)  
Chest circumference (cm)  
Abdomen 2 circumference (cm)  
Hip circumference (cm)  
Thigh circumference (cm)  
Knee circumference (cm)  
Ankle circumference (cm)  
Biceps (extended) circumference (cm)  
Forearm circumference (cm)  
Wrist circumference (cm) 

"Measurement standards are listed in Benhke and Wilmore (1974), pp. 45-48 where, for instance, the abdomen 2 circumference is measured "laterally, at the level of the iliac crests, and anteriorly, at the umbilicus."

Data Description:
The data is a real collection of 252 men with measurements of their percentage of body fat and various body circumference measurements. There are totally 16 variables, including Body Density and Body Fat Percentage.


In [1]:
library(car)
library(leaps)
library(faraway)
library(glmnet)
options(repr.plot.width=4, repr.plot.height=3)

bf <- read.csv('~/Desktop/628/module 1/BodyFat.csv', header = TRUE)
bf <- bf[ , -1]
names(bf) <- tolower(names(bf))

Loading required package: carData

Attaching package: ‘faraway’

The following objects are masked from ‘package:car’:

    logit, vif

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-16



We first check the distribution of each variables in the data. Most variables distribute normally. However, several variables have some outliers faraway from other data points. We find that **patient 39** has a weight of 363 lbs and his circumferences of *biceps, knee, hip, thigh, abdomen, chest and neck*  are far larger than anyone else. Another fun fact is that **patient 182** has 0 bodyfat with smallest circumferences of *abdomen, hip, thigh, chest, adiposity and weight* amoung all the patients.  

We consider these two patients as potential outliers in the data.

ankle 31, 86 large than expected but still within threshold

In [2]:
summary(bf)

layout(matrix(1:ncol(bf), ncol = 4))
for(i in 1:ncol(bf)){
  hist(bf[, i], breaks = 30, main = paste0('distribution of ', names(bf)[i]), xlab = names(bf)[i])
}
layout(1)

    bodyfat         density           age            weight     
 Min.   : 0.00   Min.   :0.995   Min.   :22.00   Min.   :118.5  
 1st Qu.:12.80   1st Qu.:1.041   1st Qu.:35.75   1st Qu.:159.0  
 Median :19.00   Median :1.055   Median :43.00   Median :176.5  
 Mean   :18.94   Mean   :1.056   Mean   :44.88   Mean   :178.9  
 3rd Qu.:24.60   3rd Qu.:1.070   3rd Qu.:54.00   3rd Qu.:197.0  
 Max.   :45.10   Max.   :1.109   Max.   :81.00   Max.   :363.1  
     height        adiposity          neck           chest       
 Min.   :29.50   Min.   :18.10   Min.   :31.10   Min.   : 79.30  
 1st Qu.:68.25   1st Qu.:23.10   1st Qu.:36.40   1st Qu.: 94.35  
 Median :70.00   Median :25.05   Median :38.00   Median : 99.65  
 Mean   :70.15   Mean   :25.44   Mean   :37.99   Mean   :100.82  
 3rd Qu.:72.25   3rd Qu.:27.32   3rd Qu.:39.42   3rd Qu.:105.38  
 Max.   :77.75   Max.   :48.90   Max.   :51.20   Max.   :136.20  
    abdomen            hip            thigh            knee      
 Min.   : 69.40  

ERROR: Error in replayPlot(obj): invalid graphics state


Plot with title “distribution of wrist”

### 1.2 Verify Data

#### Bodyfat

As the data discription shows, the body fat is calculated from Siri's equation: $$Body fat = \frac{495}{Density} - 450$$ 
We can verify body fat data by density. We plot a picture of body fat in the data vs calculated body fat. It's obvious that several body fat data points are quite different from the calculated one. 

There are 5 points whose differences between two types of body fat are larger than 2. We compare their body figures with other patients with similar body fat. We tend to keep the body fat in the data of **patient 48, 76, 96 and 216**. The difference in the other variables are not enough to say that the body fat value in the data is wrong. The **patient 182** has a 0 body fat in the data and -3% calculated body fat. Based on the fact this patient also has relatively smaller body figure than other people, the data of this patient may have some error or the patient is in some special condition. Thus this patient is not suitable for the model we are going to build. We exclude this point in our model.

Then we delete the density in the data, as it's figure that very hard to obtain by people themselves and impossible to use in the calculator.  

In [ ]:
bdfat <- 495/bf$density - 450
plot(bf$bodyfat, bdfat)
abline(1,1, lty = 2)

hist(bdfat - bf$bodyfat, breaks = 20)
sp_point <- which(abs(bdfat - bf$bodyfat) > 2)
sp_data <- bf[sp_point, ] 
sp_data <- cbind(bdfat[sp_point], sp_data)
sp_data

#### Adiposity(BMI)
The adiposity is calculated by the weight and height. By simillar process like above, we find **patient 42** has wrong height data. We replace the height of **patient 42** with the height calculated by his weight and adiposity. After this, the value of this patient is reasonable. The relatively small data size is also one reason that we keep this patient rather than delete him.


In [ ]:
bmi <-  (bf$weight*0.454)/(bf$height*2.54/100)^2
hist(bmi - bf$adiposity, breaks = 20)
sp_point1 <- which(abs(bmi - bf$adiposity) > 5)
sp_data1 <- cbind(bf[sp_point1 , 1:6], bmi = bmi[sp_point1], bf[sp_point1, 7:16])
sp_data1
# 42 wrong height
bf[42, ]$height <- round(sqrt((bf[42, ]$weight*0.454)/bf[42, ]$adiposity)*100/2.54, 2)

bf <- bf[ , -2]

### 1.3 Diagnose Data

We then build a linear model to check the outliers. We can see that **patient 39, 86, 221** have very large cook's distance than other points. **Patient 39** has extremly large weight. We think he should be fitted by some special model. **Patient 86** has relatively large ankle circumference than people with similar body fat. **Patient 221** has relatively large abdomen circumference than people with similar body fat. Thus we exclude **patient 39** and keep other two in the data.


To summarize, we exclued **patient 39, 182** in our model.

In [ ]:
fit1 <- lm(bodyfat ~ ., data = bf[-182, ])
summary(fit1)
plot(fit1, which = 4)
abline(h = 4/(nrow(bf) - ncol(bf)), lty = 2)

# outlierTest(fit1) # 221

bf[c(39, 86, 221), ]

## 2.Variable Selection

### 2.1 Background

There is a method to calculate body density accurately based on determining body volume by underwater submersion. But it is hard for doctors who want to and easily quickly determine a patient’s body fat percentage based on commonly available measurements to use.

Accordind to some popular books, estimating the percentage of body fat is at least a part, considered to assess body health. The books show that age, skin-fold measurements and body circumference measurements are used to estimate body fat. Additionally, online body fat percentage calculators show that among age, skin-fold measurements and body circumference measurements, "AGE", "WEIGHT","HEIGHT", "WRIST",HIP" are often used.


### 2.2 Stepwise selection

In [ ]:
regModel <- function(bf_data){
  result <- list()
  fit2 <- lm(bodyfat ~ ., data = bf_data) # without 2 outliers
  layout(matrix(1:4, ncol = 2))
  plot(fit2)
  layout(1)
  
  fit2_AIC <- step(fit2, k = 2, direction = 'backward', trace = F)
  fit2_BIC <- step(fit2, k = log(nrow(bf_data)), direction = 'backward', trace = F)

  fit_base <- lm(bodyfat ~ 1, data =  bf_data)
  base_AIC <- step(fit_base, scope = list(lower = ~ 1, upper = fit2), 
                   direction = 'forward', trace = F)
  base_BIC <- step(fit_base, scope = list(lower = ~ 1, upper = fit2), 
                   k = log(nrow(bf_data)), direction = 'forward', trace = F)
  
  fit_both <- lm(bodyfat ~ weight + thigh + height + biceps + hip + wrist, data = bf_data)
  fit_both_AIC <- step(fit_both, k = 2, scope = list(lower = ~ 1, upper = fit2), trace = F)
  fit_both_BIC <- step(fit_both, k = log(nrow(bf_data)), scope = list(lower = ~ 1, upper = fit2), trace = F)

  result <- list()
  result[['fit']] = fit2
  result[['back_AIC']] = fit2_AIC
  result[['back_BIC']] = fit2_BIC
  result[['for_AIC']] = base_AIC
  result[['for_BIC']] = base_BIC
  result[['both_AIC']] = fit_both_AIC
  result[['both_BIC']] = fit_both_BIC

  return(result)
}

sumReg <- function(fun, reg){
  for(i in 1:7){
    print(names(reg)[i])
    cat("\t")
    print(fun(reg[[i]]))
  }
}

In [ ]:
bf_data <- bf[c(-39, -182), ]
reg_with_bmi <- regModel(bf_data)
reg_without_bmi <- regModel(bf_data[-5])

sumReg(coef, reg_with_bmi)
sumReg(coef, reg_without_bmi)
sumReg(vif, reg_with_bmi)
sumReg(vif, reg_without_bmi)
sumReg(summary, reg_with_bmi)
sumReg(summary, reg_without_bmi)

### 2.3 Mallow's cp

For Mallow's Cp method, we select the subsets with minimum Cp values within different variable number $p$($p<k$, $k$ predictors in all) first. Then, we compare those Cp values with $p+1$ to get the best subset for us.

In [ ]:
dat=read.csv("~/Desktop/628/module 1/BodyFat.csv")[,-1]
dat=dat[,-2]

m2=lm(BODYFAT~.,data=dat[-c(39,42),])

X=model.matrix(m2)[,-1]
Y=dat[-c(39,42),1]
library(leaps) # for leaps()
library(faraway) # for Cpplot()
g=leaps(X,Y,nbest=1)
layout(1)
Cpplot(g)

According to the figure, we can find that the model with predictor 1, 3, 6, 7, 13 and 14 is very close to the line($Cp=p+1$), and doesn't have too many predictors. We think it is the best choice for us.

In [ ]:
cp.choice=c(2,4,7,8,14,15) 
model.cp=lm(BODYFAT~.,data=dat[-c(39,42),c(1,cp.choice)])
summary(model.cp)
print("cp_AIC:")
AIC(model.cp)
print("cp_BIC:")
BIC(model.cp)

MSE=sum((Y-predict(model.cp,newdata=dat[-c(39,42),-1]))^2)/250
print("cp_MSE")
MSE

AIC, BIC and MSE are shown above and they are small comparing with other models.

### 2.4 LASSO

In [ ]:
fit_lasso <- cv.glmnet(as.matrix(bf_data[,-1]),bf_data$bodyfat,family = "gaussian", type.measure = "deviance")
fit_lasso$lambda.1se

In [ ]:
fit.lasso <- glmnet(as.matrix(bf_data[,-1]),bf_data$bodyfat,family = "gaussian", lambda = 0.507203045204555, alpha = 1)
bodyfat_pre <- predict(fit.lasso, newx = as.matrix(bf_data[,-1]))
mse.lasso <- sum((bodyfat_pre-bf_data$bodyfat)^2)/250
mse.lasso
coef(fit.lasso)

## 3.Statistical Model

### 3.1 Simple Linear Regression

### 3.2 Ridge Regression

In [ ]:
fit_BIC <- lm(bodyfat ~ abdomen + weight + wrist, data = bf_data)


lambda <- seq(0, 10, length.out = 50)
bf_x <- model.matrix(fit_BIC)[ , -1]
bf_y <- as.matrix(bf_data[ , 1])
fit_r <- glmnet(bf_x, bf_y, alpha = 1, lambda = lambda)
fit_r_cv <- cv.glmnet(bf_x, bf_y, alpha = 1)
best_lam <- fit_r_cv$lambda.min

newx_r <- as.matrix(bf_test[ , c(8, 3, 15)])
newx <- bf_test[ , c(3, 8, 15)]
newy <- bf_test[ , 1]
pred_r <- predict(fit_r, s = best_lam, newx = newx_r, type = 'link')

print(mean((pred_r - newy)^2))
print(mean((pred_lm - newy)^2))
# not enough proof for improvement

predict(fit_r, s = best_lam, newx = newx_r, type = 'coefficients')
coef(fit_BIC)

### 3.3 Model Diagnose

## 4.Summary